# Posting

Hopefully we'll be able just to use the API, and do something like:

```python
def post_X(date,file_json):
    try:
        X_API.post(date,file_json)
        return "done"
    except:
        return "error"
```

However, X's API cost a lot of money, so we're going to have to automatically post in some other way, and likely the same for all other platforms.

## Posting on Web (X)
First, let's write a program to post on X, let's start off with just a simple post of only text. This will serve as the 'intro platform' to learn how to automatically post to any other place.



## Browser Simulation/Automation

Basically what we're aiming to do, is simulate the interactions a normal user would have with their browser in order to preform a task, in this case, posting something to twitter/x.

> We're using browser automation instead of just pure HTTP requests, because seemingly X has a complex authentication method and we cannot simply send a request to make a post.

I've decided to use *Playwright*, since it seems like a relatively good option, and has overall good reviews.

In [ ]:
# importing library
from playwright.sync_api import sync_playwright

# closes browser when done
with sync_playwright() as p:
    # open our browser
    browser = p.chromium.launch(headless=False,slow_mo=50)
    
    # getting on page
    page = browser.new_page()
    page.goto("https://practicetestautomation.com/practice-test-login/")
    
    # name = password
    # input = type of element
    page.fill('input[name="username"]',"Student")
    page.fill('input[name="password"]',"Password123")
    
    # take screenshot
    page.screenshot(path="screenshot_example.png")
    
    # id = #
    page.click('button#submit')

That was pretty simple! We simply had to select the element, fill it in, and click submit! (This doesn't work in jupyter because it is 
'asynchronous').

Now, to avoid logging in, and to save the cookies, I copied my google user data and ran chrome with this:

In [ ]:
# importing library
from playwright.sync_api import sync_playwright

# data-directory
data_dir_chrome = r"C:\Users\charl\AppData\Local\Google\Chrome\User Data Copy"
# creating chrome path
chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# closes browser when done
with sync_playwright() as p:
    # open our browser
    browser = p.chromium.launch_persistent_context(user_data_dir=data_dir_chrome,
                                                   executable_path=chrome_path,
                                                   headless=False)
    
    # getting on page
    page = browser.new_page()
    page.goto("https://x.com/")
    page.screenshot(path="Xscreenshot.png")

![pic](images/Xscreenshot.png)
(it worked, we're logged in)

Now, we'll need to:
- create a post  (click, write, submit)
- verify that everything worked
- add into pipeline

for X.

To select our button, we can do

```page.click('[(data-testid="SideNav_NewTweet_Button"]')```

which finds the specific element we want by its 'data-testid'. 

Afterwords, we simply wait, type in the tweet, and click the tweet button!

In [ ]:
# importing library
from playwright.sync_api import sync_playwright

# data-directory
data_dir_chrome = r"C:\Users\charl\AppData\Local\Google\Chrome\User Data Copy"
# creating chrome path
chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

with open("posting/tweet.txt","r") as file:
    tweet = file.read()

# closes browser when done
with sync_playwright() as p:
    # open our browser
    browser = p.chromium.launch_persistent_context(user_data_dir=data_dir_chrome,
                                                   executable_path=chrome_path,
                                                   headless=False)
    
    # getting on page
    page = browser.new_page()
    page.goto("https://x.com/")
    page.wait_for_selector('[data-testid="SideNav_NewTweet_Button"]', timeout=500)  # Waits up to 5s for the button
    page.click('[data-testid="SideNav_NewTweet_Button"]')
    
    page.wait_for_timeout(40)
    page.keyboard.type(tweet)
    page.click('button[data-testid="tweetButton"]')
    page.screenshot(path="posting/images/screenshot.png")

And this just works and tweets any file.

Now, that we can basically log onto any page and post arbitrary things, let's work with the LLM-APIs to get our custom summaries/tweets/responses.

## Generating the Summary w/ LLM's

I will be Anthropic's API for claude, because it seems to be a very capable model.

In [ ]:
import os
import anthropic

# saved in environement variable
api_pass = os.environ.get("anthropic_api_key")

with open("instructions.txt", "r") as file:
    instructions = file.read()
    
with open("article.txt", "r") as file:
    article = file.read()

client = anthropic.Anthropic(
    api_key=api_pass
)



message = client.messages.create(
    # model-type
    model="claude-3-7-sonnet-20250219",
    # system rules
    system= instructions,
    # messages received
    messages=[
        {"role":"user",
         "content":article}
    ],
    max_tokens=1000
    
)



In [ ]:
msg = message.content[0].text
print(msg)
print(message.usage.output_tokens)
with open('tweet.txt','w') as tweet:
    tweet.write(msg)

1. Python-as-a-second-language empathy: we need to understand mental models of folks coming from other languages to help them learn Python effectively.

2. The blind spot problem: Learning by Googling only addresses things that confuse you enough to search for them, leaving subtle differences unexamined.

3. Learning another language before Python is a superpower - you can understand the transition journey in ways native Python devs cannot.

4. Simple code
100
0.0225


In the instructions I simply wrote how I wanted my tweet structured; now we have our tweet and summary! We can save this to a file, and we could something.

Now, we have some things to do:
- Add the link to the post, and have this automatically download, feed to the LLM, and then add the link in X
- Be able to modify/add-to instructions in simple-to-use UI
- Port to other platforms